In [1]:
def cleanData(csv_path):
    '''
    cleans up san francisco traffic data for api processing

    Params
    csv_path: str of path to csv file
    '''
    assert isinstance(csv_path, str)
    assert csv_path[-4:] =='.csv'
    import pandas as pd

    df = pd.read_csv(csv_path)
    df = df[df['count'].notna()]
    df = df[df['count'] != '#REF!']
    df['count'] = df['count'].astype(int)
    df = df[df['count'] > 5]
    df = df[df['lat_x'].notna()]
    df = df[df['lon_x'].notna()]
    df = df[df['lat_y'].notna()]
    df = df[df['lon_y'].notna()]

    df = df.drop(columns=['Unnamed: 0'])
    df.reset_index(drop=True)
    df.to_csv('../processed_datasets/sf_cleaned.csv')

cleanData('../processed_datasets/sf_data_processed.csv')

In [ ]:
def points(start_lat,start_lng,end_lat,end_lng):
    '''
    makes an api call to route and directions to get a list of coordinates of the shortest path between two geographical coordinates

    Params
    start_lat: the origin lattitude
    start_lng: the origin longitude
    end_lat: the destination lattitude
    end_lng: the destination longitude
    '''
    import requests

    url = "https://route-and-directions.p.rapidapi.com/v1/routing"
    querystring = {"waypoints":f"{start_lat},{start_lng}|{end_lat},{end_lng}","mode":"drive"}
    headers = {
        "X-RapidAPI-Key": "INSERT-YOUR-API-KEY-HERE",
        "X-RapidAPI-Host": "route-and-directions.p.rapidapi.com"
    }
    response = requests.request("GET", url, headers=headers, params=querystring)
    return response


def getCoordinates(csv_file):
    '''
    reads a csv file of starting and ending locations, creates a csv file of points of all the paths traveled

    Params
    csv_file- str to csv file location
    '''
    assert isinstance(csv_file, str)
    import pandas as pd

    df_tester = pd.read_csv(csv_file)

    points_list = []
    for i in range(len(df_tester)):
        print(i*100/(len(df_tester)), '% Complete')

        start_lat = str(df_tester['lat_x'].loc[i])
        start_lng = str(df_tester['lon_x'].loc[i])
        end_lat = str(df_tester['lat_y'].loc[i])
        end_lng = str(df_tester['lon_y'].loc[i])

        coordinates = points(start_lat,start_lng,end_lat,end_lng)
        if coordinates.status_code == 200:
            points_json = coordinates.json()
            points_list.append(points_json)
        else:
            points_list.append("error")


    df_tester['coordinates'] = points_list
    df_tester.to_csv('../processed_datasets/sf_final.csv')

getCoordinates('../processed_datasets/sf_cleaned.csv')